# 2.2.1 節

よく使う名前を定義しておく。

空リスト

In [2]:
(define nil (list))

nil

()

`push`はリストの末尾に要素を追加する。

In [3]:
(define (push x xs)
  (if (null? xs) (list x)
      (cons (car xs) (push x (cdr xs)))))

(push 5 (list 1 2 3 4))

(1 2 3 4 5)

## Ex. 2.17
`xs`がnonemptyであることは仮定してよい。
つまり、チェック無しに`car`、`cdr`を使ってよい。

In [6]:
(define (last-pair xs)
  (if (null? (cdr xs)) xs
      (last-pair (cdr xs))))

(last-pair (list 23 72 149 34))

(34)

## Ex. 2.18
`push`のおかげで実装は楽。

このように、`xs`がnonemptyであることを仮定できないときは、
必ずそれを確認した後で、`car`、`cdr`を用いる。

In [7]:
(define (reverse xs)
  (if (null? xs) nil
      (push (car xs) (reverse (cdr xs)))))

(reverse (list 1 4 9 16 25))

(25 16 9 4 1)

## Ex. 2.19
定義というより、単に別名をつけるだけ。

（このコードをJupyter上で実行するときは`amount`を小さくすること。
Calysto Schemeでは時間がかかりすぎる。）

In [8]:
(define us-coins (list 50 25 10 5 1))
(define uk-coins (list 100 50 20 10 5 2 1 0.5))

(define (cc amount coin-values)
  (let ((no-more? null?)
        (first-denomination car)
        (except-first-denomination cdr))
    (cond ((= amount 0) 1)
           ((or (< amount 0) (no-more? coin-values)) 0)
           (else (+ (cc amount (except-first-denomination coin-values))
                    (cc (- amount (first-denomination coin-values)) coin-values))))))

(cc 10 us-coins)

4

`cc`の返り値は`coins`リストの並び順に依存しない。
どの順に並べようと、手続きは合計値が`amount`となるすべてのプロセスを生成する。
ただし、額の大きいコインを先に並べた方が、時間を節約できるだろう。

## Ex. 2.20
引数の列をリストとして受け取る機能はあるが、
その逆、リストを引数の列として与える機能は無い（見当たらない）ので、
再帰を行うために内部手続きを定義する必要がある。

`parity-filter`は`x`と同じパリティの要素をフィルタする。
実際、この定義をそのままに、`same-parity-as-x?`を一般の述語に抽象化したものが、
高階手続き`filter`である。

In [9]:
(define (same-parity x . xs)
  (define (same-parity-as-x? y)
    (= (remainder (- x y) 2) 0))
  (define (parity-filter xs)
    (cond ((null? xs) nil)
          ((same-parity-as-x? (car xs)) (cons (car xs) (iter (cdr xs))))
          (else (parity-filter (cdr xs)))))
  (cons x (parity-filter xs)))

(same-parity 1 2 3 4 5 6 7)
(same-parity 2 3 4 5 6 7)

(2 4 5 6 7)

## Ex. 2.21
穴埋めなので特にコメントは無し。
`square`は「準備」参照。

`(define square (lambda (x) (* x x)))`

In [10]:
(define (square-list items)
  (if (null? items) nil
      (cons (square (car items)) (square-list (cdr items)))))

(square-list (list 1 2 3 4))

(1 4 9 16)

In [11]:
(define (square-list items)
  (map square items))

(square-list (list 1 2 3 4))

(1 4 9 16)

1つ目の定義から`square`を抽象化すると`map`の定義になる。
その場合、`(map square items)`を展開すると1つ目の定義と同じ式が現れる。

## Ex. 2.22

In [12]:
(define (square-list items)
  (define (iter things answer)
    (if (null? things) answer
        (iter (cdr things)
              (cons (square (car things)) answer))))
  (iter items nil))

(square-list (list 1 2 3 4))

(16 9 4 1)

反転の原因は次の式

``(cons (square (car things)) answer)``

`things`の先頭から取り出した要素を、`answer`の**先頭**に追加しているので、
走査が終わる頃には要素の並びが反転してしまう。
（これはiterativeな`reverse`手続きのアイデアを与える。）

もちろん、この`cons`の引数の順序を逆にしたところで問題は解決しない。

In [13]:
(define (square-list items)
  (define (iter things answer)
    (if (null? things) answer
        (iter (cdr things)
              (cons answer (square (car things))))))
  (iter items nil))

(square-list (list 1 2 3 4))

((((() . 1) . 4) . 9) . 16)

リストを操作する手続きとして見なした際、`cons`は2つの引数に異なる型を想定している。
第一引数は、リストに加えられる**要素**であり、
第二引数は、要素を受け取る**リスト**である。
したがって、この順序を逆にして、
第一引数にリストを、第二引数に要素を与えることは、リスト操作としては意味をなさない。

正しい解決法は、`answer`に要素を**末尾**から追加していくことである。
再び、`push`手続きの出番である。

In [14]:
(define (square-list items)
  (define (iter things answer)
    (if (null? things) answer
        (iter (cdr things)
              (push (square (car things)) answer))))
  (iter items nil))

(square-list (list 1 2 3 4))

(1 4 9 16)

### 補註
転んでもタダでは起きたくない人のためのiterativeな`reverse`手続き。

In [15]:
(define (reverse xs)
  (define (iter xs ans)
    (if (null? xs) ans
        (iter (cdr xs)
              (cons (car xs) ans))))
  (iter xs nil))

(reverse (list 1 2 3 4 5))

(5 4 3 2 1)

## Ex. 2.23
複数の式を並べる必要があるので直接`if`は使えない。
`cond`の出番である。

In [16]:
(define (for-each proc xs)
  (cond ((null? xs) #t)
        (else (proc (car xs))
              (for-each proc (cdr xs)))))

(for-each (lambda (x)
            (newline)
            (display x)) (list 57 321 88))


57
321
88

#t

### 補註
複数の式をまとめて一つの式にする手続き`begin`を用いれば、`if`でも`for-each`が定義できる。

In [17]:
(define (for-each proc xs)
  (cond ((null? xs) #t)
        (else (begin (proc (car xs))
                     (for-each proc (cdr xs))))))